# BigQuery ML Model Baseline

**Learning Objectives**

1. Create baseline model with BQML
2. Evaluate the performance (i.e. RMSE) of the baseline model


## Introduction 
In this notebook, we will create a baseline model to predict the weight of a baby.  We will use BigQuery ML to build a linear babyweight prediction model with the base features and no feature engineering, yet. Once our baseline model is trained, we'll evaluate its performance on the validation set by computing the RMSE.

### Load necessary libraries

Check that the Google BigQuery library is installed and if not, install it.

In [1]:
%%bash
sudo pip freeze | grep google-cloud-bigquery==1.6.1 || \
sudo pip install google-cloud-bigquery==1.6.1

google-cloud-bigquery==1.6.1


## Verify tables exist in BigQuery

Run the following cells to verify that we previously created the dataset and data tables. If not, go back to lab [1b_prepare_data_babyweight](../solutions/1b_prepare_data.ipynb) to create them.

In [2]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM babyweight.babyweight_data_train
LIMIT 0

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,mother_race


In [3]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM babyweight.babyweight_data_eval
LIMIT 0

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,mother_race


## Create the baseline model

Next, we'll create a linear regression baseline model with no feature engineering.  We'll use this to compare our later, more complex models against.

### Train the model

When creating a BQML model, you must specify the model type (in our case linear regression) and the input label (`weight_pounds`).  Note also that we are using the training data table as the data source and we don't need BQML to split the data because we have already split it ourselves.

In [4]:
%%bigquery
CREATE OR REPLACE MODEL
    babyweight.baseline_model

OPTIONS (
    MODEL_TYPE="LINEAR_REG",
    INPUT_LABEL_COLS=["weight_pounds"],
    DATA_SPLIT_METHOD="NO_SPLIT") AS

SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    babyweight.babyweight_data_train

""



REMINDER:  The query takes several minutes to complete. After the first iteration is complete, your model (`baseline_model`) appears in the navigation panel of the [BigQuery Cloud Console](https://console.cloud.google.com/bigquery). Because the query uses a `CREATE MODEL` statement to create a model, you do not see query results.

You can observe the model as it's being trained by viewing the Model stats tab in the BigQuery web UI. As soon as the first iteration completes, the tab is updated. The stats continue to update as each iteration completes.

Once the training is done, visit the [BigQuery Cloud Console](https://console.cloud.google.com/bigquery) and look at the model that has been trained. Then, come back to this notebook.

### Examine training info from the baseline model
Even though BigQuery can automatically split the data it is given, and training on only a part of the data and using the rest for evaluation, to compare with our custom models later we wanted to decide the split ourselves so that it is completely reproducible.

NOTE: The results are also displayed in the [BigQuery Cloud Console](https://console.cloud.google.com/bigquery) under the **Evaluation** tab.

In [5]:
%%bigquery
-- Information from model training
SELECT * FROM ML.TRAINING_INFO(MODEL babyweight.baseline_model)

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,1.130197,None,None,29913


### Get evaluation statistics for the baseline_model.

After creating your model, you evaluate the performance of the regressor using the `ML.EVALUATE` function. The `ML.EVALUATE` function evaluates the predicted values against the actual data.

In [6]:
%%bigquery
SELECT
    *
FROM
    ML.EVALUATE(MODEL babyweight.baseline_model,
    (
    SELECT
        weight_pounds,
        is_male,
        mother_age,
        plurality,
        gestation_weeks
    FROM
        babyweight.babyweight_data_eval
    ))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,0.82556,1.129731,0.020255,0.671016,0.350742,0.350745


**Resource** for an explanation of the [Regression Metrics](https://towardsdatascience.com/metrics-to-evaluate-your-machine-learning-algorithm-f10ba6e38234).

### Using SQL to find the RMSE on the validation set
Since this is regression, we typically use the RMSE, but natively this is not in the output of our evaluation metrics above. However, we can simply take the `SQRT()` of the mean squared error of our loss metric from evaluation of the `baseline_model` to get RMSE.

In [7]:
%%bigquery
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL babyweight.baseline_model,
    (
    SELECT
        weight_pounds,
        is_male,
        mother_age,
        plurality,
        gestation_weeks
    FROM
        babyweight.babyweight_data_eval
    ))

,rmse
0,1.062888


## Lab Summary 
In this lab, we created a baseline model with BQML and calculated the RMSE on the validation set. 

Copyright 2020 Google LLC
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.